In [ ]:
import torch
from models import MotionTransformer
from utils.get_opt import get_opt
import numpy as np
from os.path import join as pjoin
from trainers import DDPMTrainer

encoder = MotionTransformer(
        input_feats=264,
        num_frames=360,
        num_layers=8,
        latent_dim=512,
        no_clip=False,
        no_eff=False)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
opt_path = "/home/serverai/ltdoanh/Motion_Diffusion/checkpoints/beat/test/opt.txt"
opt = get_opt(opt_path, device)
opt.do_denoise = True

mean = np.load(pjoin(opt.meta_dir, 'mean.npy'))
std = np.load(pjoin(opt.meta_dir, 'std.npy'))
trainer = DDPMTrainer(opt, encoder)
trainer.load(pjoin(opt.model_dir, opt.which_epoch + '.tar'))

trainer.eval_mode()
trainer.to(opt.device)

result_dict = {}
with torch.no_grad():
        caption = ["the first thing i like to do on weekends is relaxing and i'll go shopping if i'm not that tired since i started my job i think it's very important to get good sleep during the weekend because when you have work monday to friday the whole week you're very tired so getting a good rest is as important as completing an excellent job "]
        m_lens = torch.LongTensor([360]).to(device)
        pred_motions = trainer.generate(caption, m_lens, 264)
        motion = pred_motions[0].cpu().numpy()
        print(motion.shape)
opt.result_dir = "/home/serverai/ltdoanh/Motion_Diffusion/checkpoints"
np.save(pjoin(opt.result_dir, 'motion.npy'), motion)